# GDATABT
## Ahmet Emre Usta
## Engincan Göçer
### Hacettepe University
### 25.04.2023

In [66]:
import pickle
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix

In [67]:
#load the model
model = pickle.load(open('xgboost.pkl', 'rb'))

In [68]:
#run model on test data
df_test = pd.read_csv('test.csv')

In [69]:
df_test.head()

,UserID,UserFirstTransaction,Gender,Location,Age,isChurn,TotalPrice,TotalSales,TotalDiscount,UserLastTransaction,...,Indoor Sports,Kitchen Electronics,Male Fashion,Male Shoes,Outdoor Sports,Smart Phones,Smart Watches,Sound Systems,Sport Shoes,TVs and TV Sets
0,500234532,2011-10-12 00:00:00.000,FEMALE,ANTALYA,19,1,146.70,1,0,2017-12-17 18:08:28.150,...,0,0,0,0,0,0,0,0,0,1
1,500234752,2013-10-24 00:00:00.000,MALE,ISTANBUL,34,1,443.55,8,0,2018-02-11 19:39:53.710,...,0,0,1,4,0,2,0,0,0,1
2,500234763,2017-08-20 20:18:42.910,MALE,ADANA,39,0,917.55,7,0,2018-03-17 18:41:21.750,...,0,0,0,0,0,0,0,0,1,1
3,500234829,2009-09-01 00:00:00.000,MALE,ESKISEHIR,42,0,100.50,2,0,2017-10-23 18:51:56.450,...,0,0,0,0,0,1,0,0,0,0
4,500234895,2005-11-10 00:00:00.000,MALE,ISTANBUL,31,0,1773.90,6,0,2018-03-28 18:47:27.690,...,0,0,3,2,0,0,0,0,0,0


In [70]:
# drop UserID column
df_test = df_test.drop(['UserID'], axis=1)

# Train test split
y_test = df_test['isChurn']
X_test = df_test.drop(['isChurn'], axis=1)

le = LabelEncoder()
X_test["Gender"] = le.fit_transform(X_test["Gender"])

# one hot encoding to categorical features
X_test = pd.get_dummies(X_test, columns=['Location'])

X_test["UserFirstTransaction"] = pd.to_datetime(X_test["UserFirstTransaction"])
X_test["UserLastTransaction"] = pd.to_datetime(X_test["UserLastTransaction"])

# encode the date columns
X_test["UserFirstTransaction"] = X_test["UserFirstTransaction"].apply(lambda x: x.toordinal())
X_test["UserLastTransaction"] = X_test["UserLastTransaction"].apply(lambda x: x.toordinal())

In [71]:
X_test.head()

,UserFirstTransaction,Gender,Age,TotalPrice,TotalSales,TotalDiscount,UserLastTransaction,UserActiveDays,TotalWeb,TotalMobile,...,TVs and TV Sets,Location_ADANA,Location_ANKARA,Location_ANTALYA,Location_BURSA,Location_ESKISEHIR,Location_ISTANBUL,Location_IZMIR,Location_KAYSERI,Location_TRABZON
0,734422,0,19,146.70,1,0,736680,2258,1,0,...,1,0,0,1,0,0,0,0,0,0
1,735165,1,34,443.55,8,0,736736,1571,1,7,...,1,0,0,0,0,0,1,0,0,0
2,736561,1,39,917.55,7,0,736770,208,0,7,...,1,1,0,0,0,0,0,0,0,0
3,733651,1,42,100.50,2,0,736625,2974,0,2,...,0,0,0,0,0,1,0,0,0,0
4,732260,1,31,1773.90,6,0,736781,4521,6,0,...,0,0,0,0,0,0,1,0,0,0


In [72]:
# Evaluate the best estimator on the test set
y_pred = model.predict(X_test)
print("Test set accuracy:", accuracy_score(y_test, y_pred))
print("Test set F1 score:", f1_score(y_test, y_pred))
print("Test set ROC AUC score:", roc_auc_score(y_test, y_pred))

Test set accuracy: 0.6719330855018587
Test set F1 score: 0.294
Test set ROC AUC score: 0.5699102213795796


In [73]:
#print confusion matrix
print(confusion_matrix(y_test, y_pred))

[[1299   63]
 [ 643  147]]
